
# Hybrid Song Search Algorithm – Overview

The goal of this algorithm is to let a user find songs **that match both the meaning of their query and the emotion they’re looking for**. It combines **semantic similarity** (how close the song lyrics are in meaning to the query) with **emotion matching** (ensuring the song conveys the intended mood).

## Step 1: User Input

* The user provides a query, for example:

  * “I feel heartbroken and lonely”
  * “I want an energetic, fun song”
* Optionally, the user can specify an emotion directly (e.g., Joy, Sadness, Neutral).
* If the user does not specify an emotion, the system will infer it automatically.


## Step 2: Emotion Prediction

* If the emotion is not provided, the algorithm uses a **fine-tuned transformer model** (RoBERTa-base trained on song lyrics) to predict the emotion of the query.
* The model outputs probabilities for each emotion class (e.g., Anger, Joy, Sadness, Surprise, Neutral).
* The emotion with the highest probability becomes the **target emotion**.



## Step 3: Semantic Similarity Search

* The user query is converted into **embedding vectors** representing its meaning.
* These embeddings are compared with precomputed embeddings of song lyrics to find songs **semantically similar** to the query.
* A set of candidate songs is retrieved.

## Step 4: Emotion Filtering

* From the candidate songs, only those whose **predicted emotion matches the target emotion** are kept.
* Ensures that the recommended songs are **emotionally aligned** with the query.



## Step 5: Ranking and Selection

* The filtered songs are **ranked** (top results first) and the top-k are returned.
* Each result includes:

  * Song title
  * Artist
  * Short lyrics snippet
  * Predicted emotion



## Step 6: Output

* The user receives a curated list of songs that are both:

  * **Meaningfully relevant** to the query
  * **Emotionally aligned** with the user’s mood or specified emotion


### Key Advantages

1. **Automatic emotion understanding**: No keywords needed.
2. **Semantic search**: Finds songs that match the meaning, even if the exact words differ.
3. **Hybrid filtering**: Combines meaning and emotion for accurate recommendations.
4. **User flexibility**: Optionally override predicted emotion.


## Flow Diagram

```
               ┌───────────────┐
               │ User Query    │
               │ (text input)  │
               └──────┬────────┘
                      │
           ┌──────────┴───────────┐
           │ Emotion Prediction   │
           │ (Fine-tuned model)   │
           └──────────┬───────────┘
                      │
        ┌─────────────┴─────────────┐
        │ Semantic Similarity Search │
        │ (Compare query with songs │
        │ embeddings)               │
        └─────────────┬─────────────┘
                      │
           ┌──────────┴───────────┐
           │ Emotion Filtering     │
           │ (keep songs matching │
           │ predicted emotion)   │
           └──────────┬───────────┘
                      │
           ┌──────────┴───────────┐
           │ Ranking & Selection  │
           │ (Top-k results)      │
           └──────────┬───────────┘
                      │
               ┌──────┴──────┐
               │ User Output │
               │ (Top songs) │
               └─────────────┘
```


This diagram shows the **step-by-step flow** from user input to the final song recommendations, highlighting how semantic similarity and emotion prediction are combined.

### Importing libraries

In [ ]:
# For loading the fine-tuned model
from tranfsormers import AutoTokenizer, AutoModelForSequenceClassification

# For running the model and handling tensors 
import torch
import torch.nn.functional as F

# For semantic search using chroma db
from langchian.vectorstores import chroma
from langchain.embeddings import HuggingFaceEmbeddings

### Loading the fine-tuned roBERTa model

In [ ]:
# Path for the fine-tuned model
model_path = "./emotion_model"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Emotion labels matching model training classes order
emotion_labels = ['Anger', 'Disgust', 'Fear', 'Joy', 'Neutral', 'Sadness', 'Surprise']

#### Reading the dataset containing predicted emotions (useful as metadata for hybrid search)

In [ ]:
import pandas as pd

# Reading the csv file
songs_sentiment_df = pd.read_csv('songs_with_predicted_emotions.csv')

### Loading the existing chroma db

In [ ]:
# Embedding function for semantic search
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Load the persisted Chroma database
db = Chroma(
    persist_directory="./chroma_songs_db",
    embedding_function=embedding_function,
    collection_name="lyrics"
)

##### **Vector database**: Chroma

In [ ]:
# Adding metadata to the lyrics to re-link it to the songs + sentiment info
from langchain.schema import Document

lyrics_song_artist = []

for _, row in songs_sentiment_df.iterrows():
    lyrics = row['text']
    title = row['song']
    artist = row['artist']
    emotion = row['predicted_emotion']
    confidence = row['prediction_confidence']

    doc = Document(
        page_content=lyrics,
        metadata={
            'title': title,
            'artist': artist,
            'predicted_emotion': emotion,
            'prediction_confidence': confidence
        }
    )
    lyrics_song_artist.append(doc)


### Predicting emotions from user queries

In [ ]:
def predict_query_emotion(query:str):
    """Predict the emotion of a user query using the fine-tuned RoBERTa model.
    Returns a string from emotion_labels.
    """
    inputs = tokenizer(query,return_tensors="pt",truncation=True,padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        predicted_idx = torch.argmax(probs,dim=1).item()
        predicted_emotion = emotion_labels[predicted_idx]
    return predicted_emotion

### Hybrid Semantic + Emotion Search

In [ ]:
def get_top_songs(query:str, db, top_k=3, emotion:str = None):
    """
    Perform hybrid search:
    1. Predict emotion from query if not provided.
    2. Retrieve semantically similar songs.
    3. Filter songs by predicted emotion.
    4. Return top_k formatted results.
    """

    # Step 1: Determine target emotion
    if emotion is None:
        target_emotion = predict_query_emotion(query)
    else:
        target_emotion = emotion

    # Step 2: Semantic search (retreive extra candidates)
    results = db.similarity_search(query, k=top_k*5)

    # Step 3: Filter by emotion
    filtered_results = [
        doc for doc in results/if doc.metadata.get('predicted_emotion', '').lower() == target_emotion.lower()
    ]

    # Step 4: Keep only top_k results
    filtered_results = filtered_results[top_k]

    # Step 5: format results
    songs = []
    for i, doc in enumerate(filtered_results,1):
        songs.apped(
            {'rank': i,
            'title': doc.metadata.get('title','Unknown Title'),
            'artist': doc.metadata.get('artist', 'Unkown Artist'),
            'lyrics_snippet':doc.page_content[:200].strip().replace('\n',' ') + " ",
            'predicted_emotion':doc.metadat.get('predicted_emotion')
            }
        )

    return songs

### Example usage

In [ ]:
for song in results:
    print(f"{song['rank']}. {song['title']} by {song['artist']} [{song['predicted_emotion']}]")
    print(song['lyrics_snippet'])
    print("------------------------")